In [1]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [ ]:
%use kandy
%use dataframe


In [ ]:
fun createPlotableDateFrame(datas: List<List<Double>>, labels: List<String>):DataFrame<*>{
    val firstSize = datas[0].size
    require(datas.size == labels.size )
    require(datas.all { it.size == firstSize })
    
    return dataFrameOf(
        "datas" to datas.flatten(),
        "labels" to datas.zip(labels).map {(individualData, label_string) -> List(individualData.size){label_string} }.flatten(),
        "index" to datas.map { sublist -> List(sublist.size){it}}.flatten()
    )
}

//Test
//createPlotableDateFrame(
//    datas = listOf(
//        listOf(1.0,2.0,3.0),
//        listOf(4.0,5.0,6.0),
//        listOf(7.0,8.0,9.0),
//    ),
//    labels = listOf("first label", "second label", "third label")
//).let { 
//    print(it)
//}



In [ ]:
import kotlin.random.Random
import org.kalman.calibration.measurement.Linear2DMeasurementCalibration
import org.kalman.calibration.measurement.MeasurementCalibration

/**
 * Creating 2d linear mode for model Calibration 
 */
val linearCalibration: MeasurementCalibration = Linear2DMeasurementCalibration()
linearCalibration.iterate(100)

val plotData = createPlotableDateFrame(
    datas = listOf(
        linearCalibration.hiddenStates.map { it.maxValue },
        linearCalibration.measuredStates.map { it.maxValue }
    ),
    labels = listOf("TrueHiddeStates", "MeasuredStates")
)


plotData.plot {
    line {
        x("index")
        y("datas")
        color("labels"){
            scale = categorical("TrueHiddeStates" to Color.BLUE, "MeasuredStates" to Color.RED )
        }
    }
    layout.size = 1500 to 450
    
    layout.title = "Generate 2D Linear Data For Calibration"
}


In [ ]:
import org.apache.commons.math3.linear.ArrayRealVector
import org.apache.commons.math3.linear.Array2DRowRealMatrix
import org.apache.commons.math3.linear.RealMatrix
import org.apache.commons.math3.linear.RealVector
import org.kalman.wrapper.KalmanFilterProcess

/**
 * Creating Kalman filter and calibrate it with generated dataset.
 */
val kfProcess = KalmanFilterProcess(
    debugEnabled = true,
    states = ArrayRealVector(
        doubleArrayOf(
            50.0 //initial estimation
        )
    ),
    stateTransitionMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(1.0) // assuming the state is still
        )
    ),
    covarianceMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(200.0 * 200.0) // initial co-variance
        )
    ),
    processMatrix = Array2DRowRealMatrix(
        arrayOf(
            doubleArrayOf(600*500.0) // assuming process variance of 25
        )
    )
)


repeat(linearCalibration.hiddenStates.size){index ->
    kfProcess.predict()

    kfProcess.update(
        y = linearCalibration.measuredStates[index],
        r = linearCalibration.measurementCovariances[index]
    )
}


/**
 * plot graph 
 */

val test = createPlotableDateFrame(
    datas = listOf(
        linearCalibration.hiddenStates.map { it.maxValue },
        kfProcess.debug().map { it.y.maxValue },
        kfProcess.debug().map { it.x.maxValue }
    ),
    labels = listOf(
        "Calibration_Hidden_State", 
        "KF_Measurement",
        "KF_Estimate"
        )
)

test.plot {
    line {
        x("index")
        y("datas")
        color("labels"){
            scale = categorical(
                "Calibration_Hidden_State" to Color.RED,
                "KF_Measurement" to Color.GREY,
                "KF_Estimate" to Color.PURPLE,
                )
        }
    }
    layout.size = 1500 to 450

    layout.title = "Generate 2D Linear Data For Calibration"
}
